Decision Tree algorithm by hand because it is easy enough for me to do

# Preparing data

In [0]:
import pandas as pd


# making test data that woulnd't bore me to death
data = {
    'name' : ['Frodo', 'Boromir', 'Barca', 'Massinissa', 'Opimius','Cornelia'],
    'brute' : [False, True, True, True, False, False],
    'wise' : [False, True, True, True, False, True],
    'just' : [True, True, True, False, False, False],
    'death' : [153, 41, 66, 90, 60, 70],
    'liked' : [True, False, True, True, False, True]
}
# to think of it, using 0,1 would've been easier



data = pd.DataFrame(data)

# setting name as the index and everything else in column
data = data.set_index("name")

# if needed, columns can be manually set and modified via:
# data = data[ ['col_name1','col_name2','col_name3' ...]]
# or below is easier to medofy the order :
# neworder = [0,1,2,3,4,5]
# data =  [data.columns[i] for i in order]

# checking the data
data

,brute,wise,just,death,liked
name,,,,,
Frodo,False,False,True,153,True
Boromir,True,True,True,41,False
Barca,True,True,True,66,True
Massinissa,True,True,False,90,True
Opimius,False,False,False,60,False
Cornelia,False,True,False,70,True


# Preprocessing

Decision Tree is a supervised learning == need **Label** and **Feature**

Sinsce this is classification,

label : answers for the question. in this case, determining T/F for if they are 'liked'
feature : information used to reach the conclusion


In [0]:
label = 'liked'
# and everything else goes into features :
features = data.columns.difference([label])
features

Index(['brute', 'death', 'just', 'wise'], dtype='object')

# designing a decision tree
1. Create **condition** through **features**
2. determine important **condition** depending on criterions, such as gini impurity, MSE, etc
3. Create a tree with the important conditions

1. Creating conditions

>> segmenting each features.

>> i.e : if a feature was 'score' score<50, score<70, score<85< score<100 etc

In [0]:
# two scenarios:
# True/False given one condition
# True/False given range
def categorical_condition(data, feature, value):
  return data[feature] == value

def continuous_condition(data, feature, value):
  return data[feature] < value


# combine those two to make condition
# this function should be able to determine if it is categorical or continuous
# and call one between above two functions
def make_condition(method, feature, value):
  def call_condition(data):
    return method(data, feature, value)
  return call_condition



# try this on prepared data
# condition = make_condition(categorical_condition, 'just', True) # meaning return only those are True on 'brute'
# data[condition(data)]

# another one
condition2 = make_condition(continuous_condition, 'death', 60) #returning those who died before age 60. hurts my heart.
data[condition(data)]

,brute,wise,just,death,liked
name,,,,,
Frodo,False,False,True,153,True
Boromir,True,True,True,41,False
Barca,True,True,True,66,True


In [0]:
# using above, make function that generates conditions automatically
# so if the feature is bool (T/F) then it calls categorical
# else, (if the feature is int, float, etc) then it calls continuous

def make_condition_list(data, features):
  condition_list = {}
  
  for feature in features:
    if data[feature].dtype == bool: #if T/F data
      description = f"{feature} == True" # only need one then
      condition = make_condition(categorical_condition, feature, True) #took 'true' for example
      condition_list[description] = condition #this goes into condition
    else: # if ranged data
      value_list = data[feature].unique() #take unique values
      value_list.sort() #sort them
      value_list = value_list[1:] #taking out the smallest value. could do that as the largest one too and modify 'continuous_condition'
      
      
      
      for value in value_list:
        description = f"{feature} < {value}"
        condition = make_condition(continuous_condition, feature, value)
        condition_list[description] = condition
        
  return condition_list
    

In [0]:
# test it

condition_list = make_condition_list(data, features)
print(condition_list)


print(data[condition_list['brute == True'](data)])

{'brute == True': <function make_condition.<locals>.call_condition at 0x7efc8abfa378>, 'death < 60': <function make_condition.<locals>.call_condition at 0x7efc8abfa2f0>, 'death < 66': <function make_condition.<locals>.call_condition at 0x7efc8abfa268>, 'death < 70': <function make_condition.<locals>.call_condition at 0x7efc8abfa730>, 'death < 90': <function make_condition.<locals>.call_condition at 0x7efc8abfa840>, 'death < 153': <function make_condition.<locals>.call_condition at 0x7efc8abfae18>, 'just == True': <function make_condition.<locals>.call_condition at 0x7efc8abfa8c8>, 'wise == True': <function make_condition.<locals>.call_condition at 0x7efc8abfa598>}
            brute  wise   just  death  liked
name                                        
Boromir      True  True   True     41  False
Barca        True  True   True     66   True
Massinissa   True  True  False     90   True


# Criterion 
determines what feature is more detrimental and what is less

less potent feature will be located on lower node or will not be used at all

Classification -- Gini Impurity

regression -  mean squared error

also entroty, mean absolute error

## 1. Gini Impurity

$$
 I_{G}(p) = 1 - (p_{true})^{2} - (p_{false})^{2}
$$
$(P_{true})^{2}$ == possibility the result is true

$(P_{false})^{2}$ == possibility the result is false

The goal is to get 0, to get a perfectly divided result. means 0.5 is the worst, in case everything is half and half




In [0]:
#making gini impurity for true and false, binary classification
def gini(data, label):
  if len(data) == 0:
    return 0
  
  true_prob = data[label].mean()
  false_prob = 1 - true_prob
  
  gini = 1 - (true_prob **2) - (false_prob **2)
  
  return gini

#okay so how about multi-class?

## 2 . multi-class gini impurity
$$ {I} _{G}(p) = 1 - \sum _{i=1}^{J}{p_{i}}^{2} $$
subtract posibility^2 of each class from 1


In [0]:
#re-defining gini so that it can address everything
def gini(data, label):
  if len(data) == 0:
    return 0
  
  gini = 1 # initialize
  class_list = data[label].unique()
  
  for class_value in class_list :
    prob = len( data [data[label] == class_value] ) / len(data)
    
    gini = gini - (prob ** 2)
    
  return gini
  

In [0]:
gini(data, 'liked')

0.4444444444444445

## 3.  Entropy
$$H_{i}=-\sum _{k=1}^{n}\ {p_{i,k}}*log_{2}({p_{i,k})}$$

measures the chaos : 50/50 is therefore 0.5



### ex) H(10,20)
>> $ log\frac{10}{20} - \frac{10}{20} log\frac{10}{20} $

There isn't much difference between Entropy and Gini Impurity,

but Gini Impurity computes faster, and entropy makes more balanced tree




## 4. mean squared error

This is for Regression.
$$
 {MSE} ={\frac {1}{n}}\sum _{i=1}^{n}(y_{i} - {\hat {y_{i}}})^{2}
$$
 $y_{i} $== actual value, $\hat{y_{i}}$ == prediction
 
 
 so mean of square of real value - predicted values
 
 
 so lower MSE the better. Decision tree will evolve where returned MSE is the lowest.

In [0]:
def eval_mse(data, label):
  if len(data) == 0:
    return 0
  
  mean = data[label].mean()
  mse = ((data[label] - mean) ** 2).mean()
  
  return mse

## Get Criterion score
so there are two criterion to test : gini impurity and MSE